In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm 
import pandas as pd

data = pd.read_csv('data/area_full.csv')

family_names = data.FAMILY.unique()

n_families = len(data.FAMILY.unique())
family_key = data['FAMILY'].unique()
num_traits = 7

/home/diogro/bin/anaconda3/envs/pymc2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
family = family_key[2]
family_data = data[(data['FAMILY'] == family)]
ind_idx = family_data.ID.values
ind = ind_idx[1]
locus_number= 9
family_data[family_data['ID'] == ind].filter(regex = 'chrom4_A{}'.format(str(locus_number)))

,chrom4_A9
18,0


In [3]:
%%writefile mixed_model.py

import matplotlib.pyplot as plt
import numpy as np
import pymc as pm 
import pandas as pd

data = pd.read_csv('data/area_full.csv')

family_names = data.FAMILY.unique()

n_families = len(data.FAMILY.unique())
family_key = data['FAMILY'].unique()
num_traits = 7

# Overall mean prior
mu = pm.MvNormalCov("mu", 
                    value=np.array(data.filter(regex = 'area')).mean(axis = 0),
                    mu=np.zeros(num_traits),
                    C=np.eye(num_traits)*100.)

# Family means prior
mu_f = pm.MvNormalCov("mu_f", 
                      value=np.zeros(num_traits),
                      mu=np.zeros(num_traits),
                      C=np.eye(num_traits)*100.)

# G matrix priors, covariance matrix of family means
G = pm.WishartCov('G',
                  value=data.groupby('FAMILY').mean().filter(regex = 'area').cov(),
                  n=num_traits+1,
                  C=np.eye(num_traits)*100.)

# R matrix prior, residual within-family covariance
R =  pm.WishartCov('R',
                   value=data.filter(regex = 'area').cov() - data.groupby('FAMILY').mean().filter(regex = 'area').cov(),
                   n=num_traits+1,
                   C=np.eye(num_traits)*100.)

locus_number = 9

# Additive effect for single locus prior
a = pm.MvNormalCov("a_{}".format(str(locus_number)), 
                   value=np.zeros(num_traits),
                   mu=np.zeros(num_traits),
                   C=np.eye(num_traits)*100.)

# Dominance effect for single locus prior
d = pm.MvNormalCov("d_{}".format(str(locus_number)), 
                   value=np.zeros(num_traits),
                   mu=np.zeros(num_traits),
                   C=np.eye(num_traits)*100.)

betas = {}
lik = {}

for family in family_key:
    
    family_data = data[(data['FAMILY'] == family)]
    family_array = np.array(data[(data['FAMILY'] == family)].filter(regex = 'area'))
    
    
    betas[str(family)] = pm.MvNormalCov('betas_{}'.format(str(family)),
                                        value = family_array.mean(axis = 0),
                                        mu = mu_f,
                                        C = G)
    ind_idx = family_data.ID.values
    
    for ind in ind_idx:
        genetic_effects = a * np.array(family_data[family_data['ID'] == ind].filter(regex = 'chrom4_A{}'.format(str(locus_number)))) + \
                          d * np.array(family_data[family_data['ID'] == ind].filter(regex = 'chrom4_D{}'.format(str(locus_number)))) 
        
        ind_mean = mu + betas[str(family)] + genetic_effects
        lik[str(ind)] = pm.MvNormalCov('data_{}'.format(ind),
                                       mu = ind_mean,
                                       C = R,
                                       value = np.array(family_data[family_data['ID'] == ind].filter(regex = 'area')),
                                       observed = True)

Overwriting mixed_model.py


In [4]:
import mixed_model

In [5]:
M = pm.MCMC(mixed_model)

In [6]:
pm.graph.graph(M,format='png',path='',name='graph_mine',prog='dot')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.0647501 to fit



In [7]:
M.sample(iter=8000, burn=3000, thin=10)

 [-----------------100%-----------------] 8001 of 8000 complete in 4228.7 sec

In [9]:
M.stats()['G']['quantiles'][50] + M.stats()['R']['quantiles'][50]

array([[ 0.01232975,  0.01440133,  0.01336446,  0.03624032,  0.02618893,
         0.004233  ,  0.01949594],
       [ 0.01440133,  0.32785033,  0.10044304,  0.18527052,  0.18195184,
         0.03006503,  0.09852918],
       [ 0.01336446,  0.10044304,  0.18361409,  0.25283385,  0.1712131 ,
         0.0268048 ,  0.08825747],
       [ 0.03624032,  0.18527052,  0.25283385,  0.63744668,  0.37933567,
         0.05895053,  0.21660326],
       [ 0.02618893,  0.18195184,  0.1712131 ,  0.37933567,  0.368628  ,
         0.04770059,  0.16438188],
       [ 0.004233  ,  0.03006503,  0.0268048 ,  0.05895053,  0.04770059,
         0.01577299,  0.02659537],
       [ 0.01949594,  0.09852918,  0.08825747,  0.21660326,  0.16438188,
         0.02659537,  0.14257872]])

In [10]:
data.filter(regex = 'area').cov()

,area1,area2,area3,area4,area5,area6,area7
area1,0.012330,0.014401,0.013364,0.036240,0.026189,0.004233,0.019496
area2,0.014401,0.327850,0.100443,0.185271,0.181952,0.030065,0.098529
area3,0.013364,0.100443,0.183614,0.252834,0.171213,0.026805,0.088257
area4,0.036240,0.185271,0.252834,0.637447,0.379336,0.058951,0.216603
area5,0.026189,0.181952,0.171213,0.379336,0.368628,0.047701,0.164382
area6,0.004233,0.030065,0.026805,0.058951,0.047701,0.015773,0.026595
area7,0.019496,0.098529,0.088257,0.216603,0.164382,0.026595,0.142579


In [14]:
M.stats()['a_9']

{'95% HPD interval': array([[-0.00182932,  0.03384261,  0.02732477,  0.00097962,  0.0305813 ,
         -0.02008855,  0.03624411],
        [ 0.01970349,  0.15268855,  0.12895834,  0.20861395,  0.1622912 ,
          0.00324816,  0.11199223]]),
 'mc error': array([ 0.00044434,  0.00309074,  0.00276566,  0.00542997,  0.00353643,
         0.00054167,  0.00210745]),
 'mean': array([ 0.00981704,  0.10643262,  0.09366685,  0.14099256,  0.11744161,
        -0.00682014,  0.0836196 ]),
 'n': 500,
 'quantiles': {2.5: array([-0.00277622,  0.02734348,  0.02801898, -0.00243039,  0.03252197,
         -0.0207932 ,  0.03324085]),
  25: array([ 0.00708806,  0.09270426,  0.07866452,  0.11747454,  0.10144054,
         -0.01028961,  0.07508356]),
  50: array([ 0.01021143,  0.11520931,  0.10266603,  0.15998363,  0.12867685,
         -0.00575688,  0.09065038]),
  75: array([ 0.01330851,  0.12847491,  0.11218806,  0.17474476,  0.14091335,
         -0.00254932,  0.09847557]),
  97.5: array([ 0.01908154,  0.1510